## Proyecto Integrador del Módulo 3 de Henry. Carrera: DataScientist 

### Importación de las librerias a usar:

In [2]:
import requests
from bs4 import BeautifulSoup
import pymysql
import pandas as pd
import datetime

fecha_hoy = datetime.date.today()

### Parte del Web Scraping

In [3]:
url_base = 'https://cuspide.com/100-mas-vendidos/'    # Paso 1 obtener html
pedido = requests.get(url_base)
html_obtenido = pedido.text                           # Paso 2. html pasarlo a texto ya que si no, nos devuelve un object de requests

soup = BeautifulSoup(html_obtenido, 'html.parser')    # 3. atribuirlo a soup para 'parsear'

In [4]:
h3 = soup.find_all('h3')   # aislamos la etiqueta h3 donde se encuentran el titulo y la url

# obtenemos el titulo en una lista
titulo_obtenido =[]       

for n in h3:
    titulo = n.find('a').text.strip()
    titulo_obtenido.append(titulo)

 # Obtenemos la url en una lista
url_obtenida = []           

for url in h3:
    u = url.find('a').get('href')
    url_obtenida.append(u)

In [5]:
# Observar cantidad de libros por url
print(len(url_obtenida))

100


In [6]:
# Listas para guardar los titulos y las url's que den error
titulo_error = []
url_error = []

In [7]:
# Listas donde se guardan los precios en pesos y dolar oficial
precio_pesos = []
precio_dolar = []

for e in url_obtenida:   # por cada url, hago un request

      url_bucle = e
      pedido2 = requests.get(url_bucle)
      html = pedido2.text
      sopa_bucle = BeautifulSoup(html, 'html.parser')

      t = sopa_bucle.find('h1').text.strip()

      try:
         # obtenemos el precio en pesos
        precio = sopa_bucle.find('p', class_='price product-page-price')
        precio_texto = precio.text.strip().replace('&nbsp;', '').replace('$', '').replace('\\xa0', '').replace('.', '')
        precio2 = float(precio_texto.replace(',', '.'))
        precio_pesos.append(precio2)

           # Obtenemos el precio en dólares
        precio_dolars = sopa_bucle.find('span', style='font-size: 1.3em').text
        precio_dolars2 = float(precio_dolars.replace(',', '.'))
        precio_dolar.append(precio_dolars2)

      except:
         print(f'> El libro: {t}, no se pudo añadir \n')
         titulo_error.append(t)
         url_error.append(url_bucle)
         precio_pesos.append(None)
         precio_dolar.append(None)

> El libro: LA PSICOLOGIA DEL DINERO, no se pudo añadir 

> El libro: DESTROZA ESTE DIARIO ( A TODO COLOR ), no se pudo añadir 

> El libro: NUESTRA PARTE DE NOCHE, no se pudo añadir 

> El libro: DEJA DE SER TU, no se pudo añadir 

> El libro: EL DUELO ( CUANDO EL DOLOR SE HACE CARNE ), no se pudo añadir 

> El libro: LA ASISTENTA, no se pudo añadir 

> El libro: TAYLOR : FROM THE VAULT, no se pudo añadir 

> El libro: LA CASA DE LOS SUICIDIOS, no se pudo añadir 

> El libro: TOKIO BLUES – BOOKET VERANO 2023-2024, no se pudo añadir 

> El libro: HABITOS ATOMICOS, no se pudo añadir 



In [8]:
# Comprobar que se hayan guardado en las listas de error los datos
for i in range(len(url_error)):
    print(f'Número: {i+1}||| titulo: {titulo_error[i]} ||| url: {url_error[i]}')

Número: 1||| titulo: LA PSICOLOGIA DEL DINERO ||| url: https://cuspide.com/producto/la-psicologia-del-dinero/
Número: 2||| titulo: DESTROZA ESTE DIARIO ( A TODO COLOR ) ||| url: https://cuspide.com/producto/destroza-este-diario-a-todo-color/
Número: 3||| titulo: NUESTRA PARTE DE NOCHE ||| url: https://cuspide.com/producto/nuestra-parte-de-noche/
Número: 4||| titulo: DEJA DE SER TU ||| url: https://cuspide.com/producto/deja-de-ser-tu/
Número: 5||| titulo: EL DUELO ( CUANDO EL DOLOR SE HACE CARNE ) ||| url: https://cuspide.com/producto/el-duelo-cuando-el-dolor-se-hace-carne/
Número: 6||| titulo: LA ASISTENTA ||| url: https://cuspide.com/producto/la-asistenta/
Número: 7||| titulo: TAYLOR : FROM THE VAULT ||| url: https://cuspide.com/producto/taylor-from-the-vault/
Número: 8||| titulo: LA CASA DE LOS SUICIDIOS ||| url: https://cuspide.com/producto/la-casa-de-los-suicidios/
Número: 9||| titulo: TOKIO BLUES – BOOKET VERANO 2023-2024 ||| url: https://cuspide.com/producto/tokio-blues-booket-ve

In [9]:
# obtenemos el dolar blue
url_blue = 'https://www.clarin.com/economia/divisas-acciones-bonos?gclid=Cj0KCQiA-62tBhDSARIsAO7twbYBCdmDyspJS1SAuZ8OcXFZZKpEUJx-A93vXK2IlRpKwbhC9_-eEAIaAk6IEALw_wcB'
pedido3 = requests.get(url_blue)
html3 = pedido3.text
sopa_blue = BeautifulSoup(html3, 'html.parser')

blue = sopa_blue.find_all('div', class_='selling')
blues = []

for n in blue:
    num = n.find('span').text.strip()
    blues.append(num)

blue_verdadero = float(blues[2].replace('$', '')) 

In [10]:
# Obtenemos el precio de cada libro con respecto al dolar blue
precio_blue = []

for p in range(len(precio_pesos)):
    if precio_pesos[p] != None:
        precio_blue.append(round(precio_pesos[p]/blue_verdadero, 2))
    else:
        precio_blue.append(None)

In [11]:
''' creamos dos dataframes a partir de dos diccionarios, uno para los libros scrapeados
 con los datos completos y otra para la auditoría de errores '''

# Dict Libros scrapeados
libros_scraping = dict( 
                titulo = titulo_obtenido,
                url = url_obtenida,
                precio_pesos = precio_pesos,
                precio_dolar = precio_dolar,
                precio_dolar_blue = precio_blue,
                fecha = fecha_hoy)

# Dict Auditoria de errores
libros_errores = dict(
                titulo = titulo_error,
                url = url_error,
                fecha = fecha_hoy)

df_con_dup = pd.DataFrame(libros_scraping)
df_errores = pd.DataFrame(libros_errores)
df_sin_dup = df_con_dup.dropna()

In [12]:
''' Creamos dos csv a partir de los dataframes de libros(sin duplicados) y 
 errores (lo hago mayor mente para visualizar lo datos en un csv): '''

df_sin_dup.to_csv('Cuspide_100.csv', encoding='utf_8', sep=',', index=False)

df_errores.to_csv('Cuspide_100_errores.csv', encoding='utf_8', sep=',', index=False)

### Acá empieza la carga de los datos a MySQL:

In [24]:
#Establezco la conexión
conexion = pymysql.connect(
   host='localhost',
   user='root',
   passwd='12345678',
   db='cuspide_100'
)
cursor = conexion.cursor()

In [25]:
# Creo ambas tablas:

# Tabla libros:
cursor.execute('''CREATE TABLE IF NOT EXISTS libros (
                    id_libro INT NOT NULL AUTO_INCREMENT,
                    titulo VARCHAR(150) NOT NULL,
                    url VARCHAR(255) NOT NULL,
                    precio_pesos DECIMAL(20,2) NOT NULL,
                    precio_dolar_oficial DECIMAL(20,2) NOT NULL,
                    precio_dolar_blue DECIMAL(20,2) NOT NULL,
                    fecha DATE NOT NULL,

                    PRIMARY KEY(id_libro) 

)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_spanish_ci;''')


# Tabla auditoria_errores:
cursor.execute('''CREATE TABLE IF NOT EXISTS auditoria_errores (
                    id_error INT NOT NULL AUTO_INCREMENT,
                    titulo VARCHAR(150) NOT NULL,
                    url VARCHAR(255) NOT NULL,
                    fecha DATE NOT NULL,
               
                    PRIMARY KEY(id_error) 

)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_spanish_ci;''')

0

### Convierto las columnas de los dataframes en listas, para luego convertirlas en listas de tuplas:

### Parte tabla libros

In [26]:
# Creamos listas que contengas los datos del dataframe df_sin_dup:

titulo = [i for i in df_sin_dup['titulo']]
url = [i for i in df_sin_dup['url']]
precio = [i for i in df_sin_dup['precio_pesos']]
precio_usd_oficial = [i for i in df_sin_dup['precio_dolar']]
precio_usd_blue = [i for i in df_sin_dup['precio_dolar_blue']]
fecha = [i for i in df_sin_dup['fecha']]

In [27]:
# Convertimos a lista de tuplas
datos_libros = []

for i in range(len(titulo)):
    datos_libros.append((titulo[i],url[i],precio[i], precio_usd_oficial[i], precio_usd_blue[i], fecha[i]))

In [28]:
# Cargamos los datos en la tabla libros
cursor.executemany('''INSERT INTO libros (titulo, url, precio_pesos, precio_dolar_oficial, precio_dolar_blue, fecha)
                   VALUES (%s,%s,%s,%s,%s,%s)''', datos_libros)

90

### Parte tabla auditoria_errores:

In [29]:
# Creamos listas que contengas los datos del dataframe df_errores:

titulo_errores = [i for i in df_errores['titulo']]
url_errores = [i for i in df_errores['url']]
fecha_errores = [i for i in df_errores['fecha']]

In [30]:
# Convertimos a lista de tuplas
datos_errores = []

for i in range(len(titulo_errores)):
    datos_errores.append((titulo_errores[i], url_errores[i], fecha_errores[i]))

In [31]:
# Cargamos los datos en la tabla auditoria_errores:
cursor.executemany('''INSERT INTO auditoria_errores (titulo, url, fecha)
                   VALUES (%s,%s,%s)''', datos_errores)

10

### Hacemos el Commit y cerramos la conexión:

In [32]:
conexion.commit()

In [33]:
conexion.close()